In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Personal Color 분류 CLIP 2 - FineTuning

# 라이브러리 임포트
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install transformers

import torch
import torch.nn as nn
import clip
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import glob
from torchvision import transforms
from tqdm import tqdm
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.6 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-cuironei
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-cuironei
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=29b11edf1c7e8f5f01c189f5933b3b7c9efd1652df715cf50d2276a987b2324d
  Stored in directory: /tmp/pip-ephem-wheel-cache-uxu2f92p/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [ ]:
class PersonalColorDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        image = image.float()
        label = self.labels[idx]
        return image, label

In [ ]:
class CLIPFineTune(nn.Module):
    def __init__(self, num_classes):
        super(CLIPFineTune, self).__init__()
        self.clip_model, _ = clip.load("ViT-B/32", device="cuda", jit=False)
        self.clip_model.float()
        self.classifier = nn.Linear(512, num_classes)

        # 마지막 레이어만 fine tuning
        for name, param in self.clip_model.named_parameters():
            if "visual.transformer.resblocks.11" not in name:
                param.requires_grad = False

    def forward(self, image):
        features = self.clip_model.encode_image(image)
        outputs = self.classifier(features)
        return outputs

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')

        # Train
        model.train()
        running_loss = 0.0
        train_preds = []
        train_labels = []

        for images, labels in tqdm(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_preds.extend(predicted.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        train_acc = accuracy_score(train_labels, train_preds)

        # Val
        model.eval()
        val_preds = []
        val_labels = []

        with torch.no_grad():
            for images, labels in tqdm(val_loader):
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                val_preds.extend(predicted.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_acc = accuracy_score(val_labels, val_preds)

        print(f'Training Loss: {running_loss/len(train_loader):.4f}')
        print(f'Training Accuracy: {train_acc:.4f}')
        print(f'Validation Accuracy: {val_acc:.4f}')

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

        print('\nClassification Report (Validation):')
        print(classification_report(val_labels, val_preds,
                                 target_names=['spring', 'summer', 'fall', 'winter']))

In [6]:
def main():
    dataset_dir = '/content/drive/Othercomputers/내 노트북/personal-color-data/'
    dataset_types = ['train', 'test']
    class_folders = ['spring', 'summer', 'fall', 'winter']

    image_paths = {'train': [], 'test': []}
    labels = {'train': [], 'test': []}

    for dataset_type in dataset_types:
        for idx, class_folder in enumerate(class_folders):
            class_dir = os.path.join(dataset_dir, dataset_type, class_folder)
            for img_path in glob.glob(os.path.join(class_dir, '*.*')):
                if img_path.lower().endswith(('.jpg', '.jpeg', '.png')):
                    image_paths[dataset_type].append(img_path)
                    labels[dataset_type].append(idx)

    # 모델 선택
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Data loaders
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                           (0.26862954, 0.26130258, 0.27577711))
    ])

    train_dataset = PersonalColorDataset(image_paths['train'], labels['train'], transform)
    val_dataset = PersonalColorDataset(image_paths['test'], labels['test'], transform)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

    model = CLIPFineTune(num_classes=4).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

    # Training
    train_model(model, train_loader, val_loader, criterion, optimizer,
                num_epochs=10, device=device)

if __name__ == "__main__":
    main()

100%|████████████████████████████████████████| 338M/338M [00:03<00:00, 116MiB/s]



Epoch 1/10


100%|██████████| 15/15 [03:23<00:00, 13.54s/it]


Training Loss: 1.1467
Training Accuracy: 0.4705
Validation Accuracy: 0.5326

Classification Report (Validation):
              precision    recall  f1-score   support

      spring       0.55      0.28      0.37       214
      summer       0.45      0.57      0.50       189
        fall       0.51      0.53      0.52       266
      winter       0.61      0.71      0.66       268

    accuracy                           0.53       937
   macro avg       0.53      0.52      0.51       937
weighted avg       0.54      0.53      0.52       937


Epoch 2/10


100%|██████████| 15/15 [00:05<00:00,  2.52it/s]


...

Training Loss: 0.0585
Training Accuracy: 0.9955
Validation Accuracy: 0.4984

Classification Report (Validation):
              precision    recall  f1-score   support

      spring       0.47      0.43      0.45       214
      summer       0.44      0.55      0.49       189
        fall       0.48      0.42      0.45       266
      winter       0.59      0.59      0.59       268

    accuracy                           0.50       937
   macro avg       0.49      0.50      0.49       937
weighted avg       0.50      0.50      0.50       937


Epoch 9/10


100%|██████████| 15/15 [00:05<00:00,  2.67it/s]


Training Loss: 0.0343
Training Accuracy: 0.9992
Validation Accuracy: 0.4771

Classification Report (Validation):
              precision    recall  f1-score   support

      spring       0.45      0.29      0.35       214
      summer       0.42      0.62      0.50       189
        fall       0.45      0.47      0.46       266
      winter       0.59      0.53      0.56       268

    accuracy                           0.48       937
   macro avg       0.48      0.48      0.47       937
weighted avg       0.48      0.48      0.47       937


Epoch 10/10


100%|██████████| 15/15 [00:05<00:00,  2.50it/s]


Training Loss: 0.0467
Training Accuracy: 0.9930
Validation Accuracy: 0.5059

Classification Report (Validation):
              precision    recall  f1-score   support

      spring       0.45      0.50      0.48       214
      summer       0.47      0.50      0.49       189
        fall       0.48      0.48      0.48       266
      winter       0.62      0.53      0.57       268

    accuracy                           0.51       937
   macro avg       0.51      0.51      0.50       937
weighted avg       0.51      0.51      0.51       937



In [8]:
def main():
    dataset_dir = '/content/drive/Othercomputers/내 노트북/personal-color-data/'
    dataset_types = ['train', 'test']
    class_folders = ['spring', 'summer', 'fall', 'winter']

    image_paths = {'train': [], 'test': []}
    labels = {'train': [], 'test': []}

    for dataset_type in dataset_types:
        for idx, class_folder in enumerate(class_folders):
            class_dir = os.path.join(dataset_dir, dataset_type, class_folder)
            for img_path in glob.glob(os.path.join(class_dir, '*.*')):
                if img_path.lower().endswith(('.jpg', '.jpeg', '.png')):
                    image_paths[dataset_type].append(img_path)
                    labels[dataset_type].append(idx)

    # 모델 선택
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Data loaders
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                           (0.26862954, 0.26130258, 0.27577711))
    ])

    train_dataset = PersonalColorDataset(image_paths['train'], labels['train'], transform)
    val_dataset = PersonalColorDataset(image_paths['test'], labels['test'], transform)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

    model = CLIPFineTune(num_classes=4).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

    # Training
    train_model(model, train_loader, val_loader, criterion, optimizer,
                num_epochs=5, device=device)

if __name__ == "__main__":
    main()


Epoch 1/5


100%|██████████| 15/15 [00:05<00:00,  2.62it/s]


Training Loss: 1.1403
Training Accuracy: 0.4790
Validation Accuracy: 0.5208

Classification Report (Validation):
              precision    recall  f1-score   support

      spring       0.53      0.27      0.35       214
      summer       0.43      0.71      0.54       189
        fall       0.54      0.40      0.46       266
      winter       0.60      0.71      0.65       268

    accuracy                           0.52       937
   macro avg       0.52      0.52      0.50       937
weighted avg       0.53      0.52      0.51       937


Epoch 2/5


100%|██████████| 15/15 [00:05<00:00,  2.53it/s]


Training Loss: 0.8632
Training Accuracy: 0.6491
Validation Accuracy: 0.5496

Classification Report (Validation):
              precision    recall  f1-score   support

      spring       0.55      0.29      0.38       214
      summer       0.45      0.69      0.55       189
        fall       0.59      0.42      0.49       266
      winter       0.61      0.78      0.68       268

    accuracy                           0.55       937
   macro avg       0.55      0.55      0.53       937
weighted avg       0.56      0.55      0.53       937


Epoch 3/5


...

Training Loss: 0.6775
Training Accuracy: 0.7459
Validation Accuracy: 0.4867

Classification Report (Validation):
              precision    recall  f1-score   support

      spring       0.50      0.22      0.31       214
      summer       0.41      0.68      0.51       189
        fall       0.47      0.51      0.49       266
      winter       0.61      0.54      0.57       268

    accuracy                           0.49       937
   macro avg       0.50      0.49      0.47       937
weighted avg       0.50      0.49      0.48       937


Epoch 4/5


100%|██████████| 15/15 [00:05<00:00,  2.59it/s]


Training Loss: 0.4940
Training Accuracy: 0.8325
Validation Accuracy: 0.5304

Classification Report (Validation):
              precision    recall  f1-score   support

      spring       0.51      0.47      0.49       214
      summer       0.53      0.50      0.51       189
        fall       0.49      0.47      0.48       266
      winter       0.58      0.66      0.62       268

    accuracy                           0.53       937
   macro avg       0.53      0.53      0.53       937
weighted avg       0.53      0.53      0.53       937


Epoch 5/5


100%|██████████| 15/15 [00:05<00:00,  2.51it/s]


Training Loss: 0.3311
Training Accuracy: 0.8986
Validation Accuracy: 0.4995

Classification Report (Validation):
              precision    recall  f1-score   support

      spring       0.47      0.50      0.48       214
      summer       0.47      0.49      0.48       189
        fall       0.46      0.48      0.47       266
      winter       0.60      0.53      0.56       268

    accuracy                           0.50       937
   macro avg       0.50      0.50      0.50       937
weighted avg       0.50      0.50      0.50       937

